In [1]:
#Import relevant modules.

import pygsti
import numpy as _np

from pygsti.algorithms import fiducialselection as FS

from pygsti.construction import std1Q_XYI
#from pygsti.construction import std1Q_XY

import matplotlib.pyplot as plt
%matplotlib inline

import time

In this notebook, we'll demonstrate how to select preparation and measurement fiducials for the standard single-qubit XYI gate set.  The results are very straightforward to generalize for a non-XYI single-qubit gateset or a multiqubit gate set.

Fiducial selection is not quite as much of a "dark art" as germ selection is, but there are nonetheless equally valid choices one can make in terms of inputs to the fiducial selection function.  (The observant user will notice striking similarities between this tutorial notebook and the germ selection notebook.)  At present, we demonstrate only a subset of the functionality, but we endeavour to explain the remaining functionality as well.

In [2]:
#Import the target gate set we will select fiducials for.
#Here, it is the standard X pi/2, Y pi/2, I gate set.
gs_target = std1Q_XYI.gs_target

Let's now try to actually pick out a fiducial set.  The fiducial selection output will strongly depend on several different inputs to the function optimize_integer_fiducials_slack (argument name given in parenthesis):

0.  Whether we are attempting to compute prep. fiducials or measurement fiducials (prepOrMeas)
1.  The list of candidate germs from which the fiducials set will be chosen.  (fiducialList)
2.  The initial subset of candidate fiducials which the optimizer tests.  (initialWeights; default is None, meaning that all candidate fiducials are included in the first germ set test)
3.  The relaxation scheme used to reduce the germ list size:

    3a.  The absolute score a germ set is allowed to achieve (fixedSlack) OR
    
    3b.  The relative score a germ set is allowed to achieve (slackFrac)
    
    Typically we will use fixedSlack, and we find that fixedSlack << 1 is still sufficient.
4.  Whether or not the fiducial set *must* contain the empty string (forceEmpty)

The output should depend much more weakly on the tolerance value (tol) passed to the optimizer.

Here we demonstrate particular choices for the above inputs.  By parameter counting, one can see that these particular instances yield "optimal" results.  However, we make no claims of optimality for these choices in general (particularly when 2 or more qubits are considered).  End users are encouraged to experiment themselves with these inputs.  They are also welcome to email pygsti@sandia.gov; the pyGSTi development team will try to assist.

In [3]:
#Let's try to pick out a fiducial set.  

#First, we generate a candidate set which we'll attempt to prune.
#Here, we're looking at all* gate string sequences of maximum length 3.

#*Excluding powers and cycles.  I.e., if Gx is included as a germ, then GxGx will not be.
#Similarly, if GxGyGy is included as a germ, then GyGxGy will not be.

max_length = 3
gates = ['Gx','Gy']#We omit any identity operations here, as we don't want them in our fiducials.

testFidList = pygsti.construction.gatestring_list([()])#We want to include the empty nothing for no time as a fiducial.
testFidList += pygsti.construction.list_all_gatestrings_without_powers_and_cycles(gates,max_length)

In [4]:
start = time.time()
prepFidList1 = FS.optimize_integer_fiducials_slack(gs_target,testFidList,prepOrMeas='prep',initialWeights=None,slackFrac=1e-6)
end = time.time()
print
print "Fiducial selection completed in", end-start, "seconds."
print prepFidList1

Starting fiducial set optimization. Lower score is better.
Iteration 0: score=40, nFids=6
Moving to better neighbor
Iteration 1: score=35, nFids=5
Moving to better neighbor
Iteration 2: score=32, nFids=4
Stationary point found!
score =  32.000032
weights =  [1 0 1 0 1 1]
L1(weights) =  4

Fiducial selection completed in 0.00282096862793 seconds.
[GateString({}), GateString(Gy), GateString(GxGxGy), GateString(GxGyGy)]


In [5]:
start = time.time()
measFidList1 = FS.optimize_integer_fiducials_slack(gs_target,testFidList,prepOrMeas='meas',initialWeights=None,slackFrac=1e-6)
end = time.time()
print
print "Fiducial selection completed in", end-start, "seconds."
print measFidList1

Starting fiducial set optimization. Lower score is better.
Iteration 0: score=40, nFids=6
Moving to better neighbor
Iteration 1: score=35, nFids=5
Moving to better neighbor
Iteration 2: score=32, nFids=4
Stationary point found!
score =  32.000032
weights =  [1 1 0 0 1 1]
L1(weights) =  4

Fiducial selection completed in 0.00277090072632 seconds.
[GateString({}), GateString(Gx), GateString(GxGxGy), GateString(GxGyGy)]


/Users/kmrudin/pyGSTi/packages/pygsti/algorithms/fiducialselection.py:236: RuntimeWarning: divide by zero encountered in divide
  score = numFids * _np.sum(1./_np.linalg.eigvalsh(scoreSqMx))


In [ ]:
#Don't worry if the above returns a divide by zero warning; this just means one of the tested cases was *really* bad.

In [6]:
print "We have selected our preparation fiducials to be:"
for fid in prepFidList1:
    print '\t',fid

We have selected our preparation fiducials to be:
	{}
	Gy
	GxGxGy
	GxGyGy


In [7]:
print "We have selected our measurement fiducials to be:"
for fid in measFidList1:
    print '\t',fid

We have selected our measurement fiducials to be:
	{}
	Gx
	GxGxGy
	GxGyGy


In [15]:
FS.test_fiducial_list(gs_target,prepFidList1,'prep',returnSpectrum=True)

(True, array([ 18.24621125,   8.        ,   4.        ,   1.75378875]))

In [13]:
FS.test_fiducial_list(gs_target,measFidList1,'meas',returnSpectrum=True)

(True, array([ 18.24621125,   8.        ,   4.        ,   1.75378875]))

In [12]:
#Note that in practice, we use a set of six fiducials both for prep and measure (stored as std1Q_XYI.fiducials).
#This is for greater numerical stability; for single-qubit GST, we recommend 6 preparation and 6 measurement fiducials,
#as the added cost is not too great, and this provides prep and measure fiducials corresponding to all 6 antipodal
#points on the Bloch sphere, for a nice level of symmetry.

#However, for multiqubit GST, experimental resources become more valuable, and we recommend simply directly using 
#the outputs of the fiducial selection code.